In [2]:
!pip install pymongo

   ---------------------------------------- 0.0/898.7 kB ? eta -:--:--
   --------------------------------------- 898.7/898.7 kB 13.5 MB/s eta 0:00:00


In [3]:
!pip install pymongo[srv]

In [4]:
!pip install dnspython

In [5]:
!pip install tweepy

In [6]:
!pip install twitter

In [32]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser
import pandas as pd

In [35]:
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

BEARER_TOKEN = config['mytwitter']['bearer_token']

In [41]:
user = client.get_user(username="ishowspeedsui")
print(user.data.id)

1040214455525797888


In [42]:
user = client.get_user(username="MrBeast")
print(user.data.id)

2455740283


In [11]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/user-tweets?user=1040214455525797888&count=80", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
ee = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[2].get("entries", [])
print(len(ee))

19


In [12]:
import json
parsed = json.loads(data)

important_info = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[2].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal tweet with no thread
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        
        # The TimelineTimelineModule structure -- tweet with a thread (1 more tweet)
        elif "items" in entry["content"]:
            items = entry["content"]["items"]
            if len(items) == 2:
                tweet = items[0]["item"]["itemContent"]["tweet_results"]["result"]
            else:
                continue  # skip if "items" is empty or any other size, indicating this entry is not a user tweet
        
        else:
            continue  # skip if neither structure exists
            
        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        like_count = legacy.get("favorite_count", 0) 
        quote_count = legacy.get("quote_count", 0)
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        media_url = None
        media_type = None
        if "extended_entities" in legacy:
            media_list = legacy["extended_entities"].get("media", [])
            if media_list:
                media = media_list[0]
                media_type = media.get("type")
                media_url = media.get("media_url_https")

        important_info.append({
            "created_at": created_at,
            "text": text,
            "media_type": media_type,
            "media_url": media_url,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print(f"🎞️ Media Type : {tweet.get('media_type')}")
    print(f"🌐 Media URL  : {tweet.get('media_url')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Sat May 31 22:49:09 +0000 2025
📝 Text       : UCL⚽️ https://t.co/p3kb2t7xUl
👀 Views      : 3387208
❤️ Likes      : 292323
💬 Replies    : 771
🔁 Retweets   : 8456
🔖 Quotes     : 143
🎞️ Media Type : photo
🌐 Media URL  : https://pbs.twimg.com/media/GsT-9VSWUAAztV0.jpg
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sat Apr 12 18:24:51 +0000 2025
📝 Text       : Thank You China &amp; Mongolia🇨🇳🇭🇰🇲🇳 https://t.co/pEGTeb2PCR
👀 Views      : 19336492
❤️ Likes      : 408414
💬 Replies    : 2688
🔁 Retweets   : 19616
🔖 Quotes     : 1138
🎞️ Media Type : photo
🌐 Media URL  : https://pbs.twimg.com/media/GoWsjyAXAAAFF9H.jpg
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Fri Apr 11 13:34:44 +0000 2025
📝 Text       : I opened the wrong door 🚪
@ksi watched the chaos unfold 🤯
Eubank vs Benn — when the game ends, the real battle begins, live from London 🎮🥊

📍Tottenham Hotspur Stadium
🗓️26 April, 2025

Buy 

In [69]:
import pandas as pd

# Convert the list of dicts to a DataFrame
df = pd.DataFrame(important_info)

# Export to CSV
df.to_csv("ishowspeed_tweets.csv", index=False, encoding="utf-8")

print("Tweets exported to ishowspeed_tweets.csv")

Tweets exported to ishowspeed_tweets.csv


In [117]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/user-tweets?user=2455740283&count=80", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[2].get("entries", [])
print(len(entries))

18


In [118]:
import json
parsed = json.loads(data)

important_info = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[2].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal tweet with no thread
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        
        # The TimelineTimelineModule structure -- tweet with a thread (1 more tweet)
        elif "items" in entry["content"]:
            items = entry["content"]["items"]
            if len(items) == 2:
                tweet = items[0]["item"]["itemContent"]["tweet_results"]["result"]
            else:
                continue  # skip if "items" is empty or any other size, indicating this entry is not a user tweet
        
        else:
            continue  # skip if neither structure exists
            
        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        like_count = legacy.get("favorite_count", 0) 
        quote_count = legacy.get("quote_count", 0)
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        media_url = None
        media_type = None
        if "extended_entities" in legacy:
            media_list = legacy["extended_entities"].get("media", [])
            if media_list:
                media = media_list[0]
                media_type = media.get("type")
                media_url = media.get("media_url_https")

        important_info.append({
            "created_at": created_at,
            "text": text,
            "media_type": media_type,
            "media_url": media_url,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print(f"🎞️ Media Type : {tweet.get('media_type')}")
    print(f"🌐 Media URL  : {tweet.get('media_url')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Sat Jun 07 17:52:26 +0000 2025
📝 Text       : I took my fiancé on dates ranging from $1 all the way up to $500,000 in the new video! Madison Beer even went on a date with Nolan 😭 https://t.co/Rj1i4lzo0u
👀 Views      : 3197668
❤️ Likes      : 54759
💬 Replies    : 1940
🔁 Retweets   : 1320
🔖 Quotes     : 272
🎞️ Media Type : photo
🌐 Media URL  : https://pbs.twimg.com/media/Gs2-Lt0WcAAHNRc.jpg
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Mon Jun 02 22:35:24 +0000 2025
📝 Text       : Want a tour of the city for Beast Games season 2 and all the massive sets we’re building?? I’m going to personally show around people that donate  $100,000 to Beast Philanthropy! I have some big charity projects I want to fund so I think it’s a win/win ❤️

https://t.co/MHJ9sTef2d
👀 Views      : 1063495
❤️ Likes      : 11333
💬 Replies    : 1587
🔁 Retweets   : 747
🔖 Quotes     : 61
🎞️ Media Type : None
🌐 Media URL  : None
-------------------------

In [119]:
import pandas as pd

# Convert the list of dicts to a DataFrame
df = pd.DataFrame(important_info)

# Export to CSV
df.to_csv("mrbeast_tweets.csv", index=False, encoding="utf-8")

print("Tweets exported to mrbeast_tweets.csv")

Tweets exported to mrbeast_tweets.csv


In [16]:
import http.client

conn1 = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn1.request("GET", "/search-v2?type=Top&count=20&query=ishowspeed", headers=headers)

res1 = conn1.getresponse()
data1 = res1.read()

# Parse JSON print length of "entries"
parsed = json.loads(data1)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [17]:
import json
parsed = json.loads(data1)

important_info1 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal tweet with no thread
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        
        else:
            continue  # skip if structure do not exists
            
        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        media_url = None
        media_type = None
        if "extended_entities" in legacy:
            media_list = legacy["extended_entities"].get("media", [])
            if media_list:
                media = media_list[0]
                media_type = media.get("type")
                media_url = media.get("media_url_https")

        important_info1.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "media_type": media_type,
            "media_url": media_url,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info1, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print(f"🎞️ Media Type : {tweet.get('media_type')}")
    print(f"🌐 Media URL  : {tweet.get('media_url')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Fri Jun 06 09:23:55 +0000 2025
📝 Text       : https://t.co/PHQLUB5X7C
🌍 Language   : zxx
👀 Views      : 2203648
❤️ Likes      : 79575
💬 Replies    : 1190
🔁 Retweets   : 4461
🔖 Quotes     : 108
🎞️ Media Type : photo
🌐 Media URL  : https://pbs.twimg.com/media/GswAMVda4AAeBV2.jpg
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Thu Jun 05 21:32:24 +0000 2025
📝 Text       : iShowSpeed launched himself into his setup after achieving 40M subs on YouTube 😭 https://t.co/hgFidBSNtT
🌍 Language   : en
👀 Views      : 10151270
❤️ Likes      : 111677
💬 Replies    : 1217
🔁 Retweets   : 3443
🔖 Quotes     : 108
🎞️ Media Type : video
🌐 Media URL  : https://pbs.twimg.com/amplify_video_thumb/1930734590966013952/img/-xs79-KrXW8VzxA9.jpg
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Fri Jun 06 15:06:04 +0000 2025
📝 Text       : iShowSpeed understood what his Brazilian friend, Receba, was trying to say wi

In [92]:
# First collection of tweets
import pandas as pd

# Convert the list of dicts to a DataFrame
df1 = pd.DataFrame(important_info1)

# Export to CSV
df1.to_csv("keyword_ishowspeed_tweets.csv", index=False, encoding="utf-8")

print("Tweets exported to keyword_ishowspeed_tweets.csv")

Tweets exported to keyword_ishowspeed_tweets.csv


In [19]:
# Second collection of tweets
import pandas as pd

# Load existing CSV
existing_df = pd.read_csv("keyword_ishowspeed_tweets.csv")

# Convert new data to DataFrame
new_df = pd.DataFrame(important_info1)

# Ensure all necessary columns exist in new_df and add missing ones with empty values
for col in existing_df.columns:
    if col not in new_df.columns:
        new_df[col] = None  

# Ensure column order matches
new_df = new_df[existing_df.columns]

# Append to existing DataFrame
updated_df = pd.concat([existing_df, new_df], ignore_index=True)

# Save back to CSV
updated_df.to_csv("keyword_ishowspeed_tweets.csv", index=False, encoding="utf-8")

print("New tweets appended to keyword_ishowspeed_tweets.csv")

New tweets appended to keyword_ishowspeed_tweets.csv


C:\Users\User\AppData\Local\Temp\ipykernel_7684\3239169955.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([existing_df, new_df], ignore_index=True)


In [29]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1928946881184346278&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))
#print(json.dumps(entries, indent=4))

22


In [85]:
# Get quotes from ishowspeed's tweets
parsed = json.loads(data)

important_info = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Mon Dec 30 09:57:55 +0000 2024
📝 Text       : Speed and Kai after seeing their chats spamming “doesnt count” https://t.co/4SOcv9GyRf
🌍 Language   : en
👀 Views      : 989
❤️ Likes      : 6
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Mon Dec 30 06:34:48 +0000 2024
📝 Text       : https://t.co/UeYM6LAQl5
🌍 Language   : zxx
👀 Views      : 204
❤️ Likes      : 1
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Mon Dec 30 06:32:11 +0000 2024
📝 Text       : Even after the game they was still locked in 😭😭 https://t.co/Dv1eN68Wuv
🌍 Language   : en
👀 Views      : 184
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Tue Dec 31 16:08:19 +0000 2024
📝 Text       : https://t.co/qku5za9y

In [86]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1911123362484814166&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [87]:
parsed = json.loads(data)

important_info1 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info1.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info1, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Mon Apr 14 17:33:08 +0000 2025
📝 Text       : 🇲🇳 https://t.co/0WNRFSBukh
🌍 Language   : qme
👀 Views      : 3116
❤️ Likes      : 59
💬 Replies    : 11
🔁 Retweets   : 9
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sat May 03 12:08:02 +0000 2025
📝 Text       : speed na mongólia foi top 10 momentos épicos do cinema pqp
🌍 Language   : pt
👀 Views      : 56
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Sun Apr 13 08:05:48 +0000 2025
📝 Text       : Mispronouncing player names on FIFA sent this young man on one of the most productive trajectories ever rather than being just another mediocre hip hop media streamer
🌍 Language   : en
👀 Views      : 876
❤️ Likes      : 6
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created 

In [88]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1910687966290419764&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [89]:
parsed = json.loads(data)

important_info2 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info2.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info2, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Fri Apr 11 14:56:12 +0000 2025
📝 Text       : WTF did I just watch LOL

HE'S IN https://t.co/GPimPRKjdk
🌍 Language   : en
👀 Views      : 62119
❤️ Likes      : 1978
💬 Replies    : 15
🔁 Retweets   : 106
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sat Apr 12 14:40:58 +0000 2025
📝 Text       : All this and @KSI selfish ass can’t do a door giveaway🤦🏿‍♂️
🌍 Language   : en
👀 Views      : 1191
❤️ Likes      : 8
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : None
📝 Text       : 
🌍 Language   : 
👀 Views      : 0
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Fri Apr 11 22:13:09 +0000 2025
📝 Text       : All that money spent on the commercial, and only get 9K likes on Speed's account, compared to the tens of t

In [90]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1909883792883413235&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [91]:
parsed = json.loads(data)

important_info3 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info3.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info3, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Wed Apr 09 12:26:40 +0000 2025
📝 Text       : I can’t lie this guy is a sporting marvel - I’ve seen him do pretty much everything perfectly.
🌍 Language   : en
👀 Views      : 795
❤️ Likes      : 10
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Wed Apr 09 10:21:21 +0000 2025
📝 Text       : Knowing this freak he'd probably be able to run straight up
🌍 Language   : en
👀 Views      : 1885
❤️ Likes      : 38
💬 Replies    : 0
🔁 Retweets   : 3
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Wed Apr 09 11:08:04 +0000 2025
📝 Text       : One thing I like about you is that you are one of the most daring people on the internet.
🌍 Language   : en
👀 Views      : 2151
❤️ Likes      : 11
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : W

In [92]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1906233598879129990&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [93]:
parsed = json.loads(data)

important_info4 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info4.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info4, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Mon Mar 31 22:04:03 +0000 2025
📝 Text       : Cudze chwalicie swego n znacie https://t.co/irDiikgaQI
🌍 Language   : pl
👀 Views      : 464
❤️ Likes      : 8
💬 Replies    : 3
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Wed Apr 02 20:10:20 +0000 2025
📝 Text       : speed r u still down to coach me on fortnite https://t.co/i5AK6A2iYv
🌍 Language   : en
👀 Views      : 1895
❤️ Likes      : 85
💬 Replies    : 6
🔁 Retweets   : 15
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Tue Apr 01 04:09:12 +0000 2025
📝 Text       : How did China build this but a fart can knock over a skyscraper
🌍 Language   : en
👀 Views      : 757
❤️ Likes      : 52
💬 Replies    : 1
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : None
📝 Text       : 
🌍 Language   : 
👀 Views      : 0
❤️ Li

In [94]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1905607632272195847&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [95]:
parsed = json.loads(data)

important_info5 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info5.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info5, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Sat Mar 29 08:27:12 +0000 2025
📝 Text       : 🤣🤣🤣🤣😭😭😭
🌍 Language   : art
👀 Views      : 664
❤️ Likes      : 4
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sun Mar 30 14:59:55 +0000 2025
📝 Text       : I really want to make money so i can just be doing anything lmao
🌍 Language   : en
👀 Views      : 1124
❤️ Likes      : 14
💬 Replies    : 0
🔁 Retweets   : 3
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Sun Mar 30 15:25:42 +0000 2025
📝 Text       : I wish I had 1/5 of this guy's extroversion 😭
🌍 Language   : en
👀 Views      : 442
❤️ Likes      : 7
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Sat Mar 29 18:20:02 +0000 2025
📝 Text       : 气贯长虹 https://t.co/wBizHJ2zcA
🌍 Language   : zh
👀 Views      : 827
❤️ Likes      : 7

In [96]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1902781952811311342&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [97]:
parsed = json.loads(data)

important_info6 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info6.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info6, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Thu Mar 20 19:58:17 +0000 2025
📝 Text       : This Nigga from Cincinnati 😂🔥🔥🔥
🌍 Language   : en
👀 Views      : 1313
❤️ Likes      : 6
💬 Replies    : 1
🔁 Retweets   : 3
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : None
📝 Text       : 
🌍 Language   : 
👀 Views      : 0
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Thu Mar 20 18:48:21 +0000 2025
📝 Text       : 🐐🔥 @DICKS  @ishowspeedsui @TomBrady @KDTrey5 @TravisHunterJr @sunisalee_
🌍 Language   : qme
👀 Views      : 387
❤️ Likes      : 3
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : None
📝 Text       : 
🌍 Language   : 
👀 Views      : 0
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------

In [98]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1902031381170929955&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [99]:
parsed = json.loads(data)

important_info7 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info7.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info7, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Wed Mar 19 22:03:10 +0000 2025
📝 Text       : É normal não acompanhar esse cara? Raramente acompanho youtubers ou streamers gringos 

A galera dos comentários e RT tudo ''UAAAU SPEED VAI ACABAR COM AS PROPAGANDAS ANTICOMUNISTAS CHINESA GALERA, CHUPA 🇹🇼'' https://t.co/31jFVmlgjE
🌍 Language   : pt
👀 Views      : 728
❤️ Likes      : 11
💬 Replies    : 1
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : None
📝 Text       : 
🌍 Language   : 
👀 Views      : 0
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Thu Mar 20 04:55:31 +0000 2025
📝 Text       : https://t.co/AulWYISAH7
🌍 Language   : zxx
👀 Views      : 933
❤️ Likes      : 7
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Wed Mar 19 23:56:08 +

In [100]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1898440034954653908&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [101]:
parsed = json.loads(data)

important_info8 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info8.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info8, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Sun Mar 09 18:59:09 +0000 2025
📝 Text       : Congrats on Speed being Bulgarian cup champion https://t.co/bMBeVoJCvI
🌍 Language   : en
👀 Views      : 532
❤️ Likes      : 22
💬 Replies    : 1
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sat Mar 08 20:01:08 +0000 2025
📝 Text       : https://t.co/IQYS7sKX9p
🌍 Language   : zxx
👀 Views      : 227
❤️ Likes      : 2
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Sat Mar 08 18:55:47 +0000 2025
📝 Text       : Pdt ce temps Harry Kane c toujours 0 trophée hein
🌍 Language   : fr
👀 Views      : 212
❤️ Likes      : 2
💬 Replies    : 1
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Sat Mar 08 19:06:57 +0000 2025
📝 Text       : Two goals and both of them are pens. He is truly the next 

In [102]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1885893645066449286&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [103]:
parsed = json.loads(data)

important_info9 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info9.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info9, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Fri Mar 28 14:30:58 +0000 2025
📝 Text       : It's really Isagi https://t.co/3TvCpsyIir
🌍 Language   : en
👀 Views      : 4823
❤️ Likes      : 277
💬 Replies    : 5
🔁 Retweets   : 37
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Mon Feb 03 14:34:17 +0000 2025
📝 Text       : WWEでネックブレースしてると逆に無事なんだなと安心します
🌍 Language   : ja
👀 Views      : 603
❤️ Likes      : 5
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Tue Feb 04 06:59:08 +0000 2025
📝 Text       : “I guess you are all wondering how I got here right, well it all started when Triple H invited me to stream at the #RoyalRumble……” #WWERaw
🌍 Language   : en
👀 Views      : 414
❤️ Likes      : 1
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Mon Feb 03 21:37:16 +0000 2025
📝 Tex

In [104]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1885886441105903691&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [105]:
parsed = json.loads(data)

important_info10 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info10.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info10, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Mon Feb 03 23:58:54 +0000 2025
📝 Text       : https://t.co/YKkwsgEyyl
🌍 Language   : zxx
👀 Views      : 484
❤️ Likes      : 15
💬 Replies    : 5
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Tue Feb 04 15:09:22 +0000 2025
📝 Text       : Stop that  @ishowspeedsui 🤣🤣 https://t.co/5beI8YTstC
🌍 Language   : en
👀 Views      : 284
❤️ Likes      : 6
💬 Replies    : 1
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Mon Feb 03 04:43:41 +0000 2025
📝 Text       : Holy shit he bout took his head off
🌍 Language   : en
👀 Views      : 292
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Mon Feb 03 07:22:46 +0000 2025
📝 Text       : 😭💀💀
🌍 Language   : art
👀 Views      : 269
❤️ Likes      : 1
💬 Replies    : 1
🔁 Retweets   

In [106]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1881819385142366292&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [107]:
parsed = json.loads(data)

important_info11 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info11.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info11, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Wed Jan 22 02:28:11 +0000 2025
📝 Text       : Acá haciéndole la torta de cumpleaños a #Speed diseñadas por mi novio @angievalentinit jujuuu 

#twitch #messi&gt;ronaldo https://t.co/xnHWyrZsRz
🌍 Language   : es
👀 Views      : 37878
❤️ Likes      : 1164
💬 Replies    : 10
🔁 Retweets   : 32
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Tue Jan 21 21:49:21 +0000 2025
📝 Text       : Ya sabes donde venir la próxima https://t.co/3YuZoyA1HE
🌍 Language   : es
👀 Views      : 3247
❤️ Likes      : 22
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Tue Jan 21 22:05:13 +0000 2025
📝 Text       : https://t.co/c02MxhQ7Lw
🌍 Language   : zxx
👀 Views      : 3664
❤️ Likes      : 20
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Wed Jan 22 01

In [108]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1881752334952538298&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [109]:
parsed = json.loads(data)

important_info12 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info12.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info12, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Tue Jan 21 17:30:20 +0000 2025
📝 Text       : https://t.co/zNSQKhiJyA
🌍 Language   : zxx
👀 Views      : 469
❤️ Likes      : 14
💬 Replies    : 1
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Tue Jan 21 18:44:39 +0000 2025
📝 Text       : https://t.co/eC2w0Fawxm
🌍 Language   : zxx
👀 Views      : 511
❤️ Likes      : 11
💬 Replies    : 0
🔁 Retweets   : 1
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Tue Jan 21 18:06:00 +0000 2025
📝 Text       : *Speed llega a argentina*

Milica apareciendo de la nada en el stream: https://t.co/3PnSCEUTdc
🌍 Language   : es
👀 Views      : 75639
❤️ Likes      : 854
💬 Replies    : 0
🔁 Retweets   : 33
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Tue Jan 21 17:24:21 +0000 2025
📝 Text       : https://t.co/6qqCpcijbR
🌍 Language   : zxx
👀 Views   

In [110]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1877764124693668061&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [111]:
parsed = json.loads(data)

important_info13 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info13.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info13, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Sun Jan 12 06:45:50 +0000 2025
📝 Text       : NOOO!!!!! DO NOT GO TO ARGENTINA!!!!!!!!! https://t.co/YN36FhV8g0
🌍 Language   : en
👀 Views      : 396
❤️ Likes      : 32
💬 Replies    : 1
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Sun Jan 12 04:00:09 +0000 2025
📝 Text       : tenemos una misión https://t.co/lL7ILXjV3N
🌍 Language   : es
👀 Views      : 465
❤️ Likes      : 19
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Sat Jan 11 15:48:35 +0000 2025
📝 Text       : Cada vez que alguien se refiere a Colombia como Columbia un hada muere
🌍 Language   : es
👀 Views      : 901
❤️ Likes      : 14
💬 Replies    : 0
🔁 Retweets   : 3
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Sun Jan 12 13:24:19 +0000 2025
📝 Text       : Why was no one fact ch

In [112]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1876042237441880332&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [113]:
parsed = json.loads(data)

important_info14 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info14.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info14, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Tue Jan 07 18:09:46 +0000 2025
📝 Text       : https://t.co/hNAivhUEum
🌍 Language   : zxx
👀 Views      : 120
❤️ Likes      : 2
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Mon Jan 06 08:31:19 +0000 2025
📝 Text       : https://t.co/9cKhoLXkUy
🌍 Language   : zxx
👀 Views      : 92
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Mon Jan 06 08:47:47 +0000 2025
📝 Text       : TOMORROW I WILL BEAT TRTF2!!! https://t.co/ltvt2ukalQ
🌍 Language   : en
👀 Views      : 467
❤️ Likes      : 23
💬 Replies    : 0
🔁 Retweets   : 5
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Sun Jan 05 23:43:33 +0000 2025
📝 Text       : Le jeu d’une vie !
🌍 Language   : fr
👀 Views      : 129
❤️ Likes      : 1
💬 Replies    : 0
🔁 Retweets

In [114]:
import http.client

conn = http.client.HTTPSConnection("twitter241.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e4b2ab9554msheab82393f002853p1b315djsnbeb0afb182bc",
    'x-rapidapi-host': "twitter241.p.rapidapi.com"
}

conn.request("GET", "/quotes?pid=1873617282079101167&count=40", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse JSON print length of "entries"
parsed = json.loads(data)
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
print(len(entries))

22


In [115]:
parsed = json.loads(data)

important_info15 = []

# Navigate through tweet entries
entries = parsed.get("result", {}).get("timeline", {}).get("instructions", [])[0].get("entries", [])
for entry in entries:
    try:
        # The TimelineTimelineItem structure -- normal quote structure
        if "itemContent" in entry["content"]:
            tweet = entry["content"]["itemContent"]["tweet_results"]["result"]
        else:
            continue  # skip if structure is different from the normal structure   

        views_count = tweet.get("views", {}).get("count", 0) 
        legacy = tweet.get("legacy", {})

        created_at = legacy.get("created_at")
        text = legacy.get("full_text", "") 
        language = legacy.get("lang", "")
        like_count = legacy.get("favorite_count", 0) 
        reply_count = legacy.get("reply_count", 0)
        retweet_count = legacy.get("retweet_count", 0)

        important_info15.append({
            "created_at": created_at,
            "text": text,
            "language": language,
            "views_count": views_count,
            "quote_count": quote_count,
            "like_count": like_count,
            "reply_count": reply_count,
            "retweet_count": retweet_count
        })

    except Exception as e:
        continue

# Print results
for i, tweet in enumerate(important_info15, start=1):
    print(f"📢 Tweet #{i}")
    print(f"🕒 Created at : {tweet.get('created_at')}")
    print(f"📝 Text       : {tweet.get('text')}")
    print(f"🌍 Language   : {tweet.get('language')}")
    print(f"👀 Views      : {tweet.get('views_count')}")
    print(f"❤️ Likes      : {tweet.get('like_count')}")
    print(f"💬 Replies    : {tweet.get('reply_count')}")
    print(f"🔁 Retweets   : {tweet.get('retweet_count')}")
    print(f"🔖 Quotes     : {tweet.get('quote_count')}")
    print("-" * 60)

📢 Tweet #1
🕒 Created at : Mon Dec 30 09:57:55 +0000 2024
📝 Text       : Speed and Kai after seeing their chats spamming “doesnt count” https://t.co/4SOcv9GyRf
🌍 Language   : en
👀 Views      : 989
❤️ Likes      : 6
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #2
🕒 Created at : Mon Dec 30 06:34:48 +0000 2024
📝 Text       : https://t.co/UeYM6LAQl5
🌍 Language   : zxx
👀 Views      : 204
❤️ Likes      : 1
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #3
🕒 Created at : Mon Dec 30 06:32:11 +0000 2024
📝 Text       : Even after the game they was still locked in 😭😭 https://t.co/Dv1eN68Wuv
🌍 Language   : en
👀 Views      : 184
❤️ Likes      : 0
💬 Replies    : 0
🔁 Retweets   : 0
🔖 Quotes     : 108
------------------------------------------------------------
📢 Tweet #4
🕒 Created at : Tue Dec 31 16:08:19 +0000 2024
📝 Text       : https://t.co/qku5za9y

In [116]:
import pandas as pd

all_quotes = [important_info, important_info1, important_info2, important_info3, important_info4, important_info5, important_info6, important_info7,
             important_info8, important_info9, important_info10, important_info11, important_info12, important_info13, important_info14, important_info15] 

# Create ExcelWriter
with pd.ExcelWriter("ishowspeed_quotes.xlsx", engine="openpyxl") as writer:
    for i, quotes in enumerate(all_quotes, start=1):
        # Convert list of dicts to DataFrame
        df = pd.DataFrame(quotes)
        # Write to sheet named "Tweet_1", "Tweet_2", etc.
        sheet_name = f"Tweet_{i}"
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ All quote data has been written to ishowspeed_quotes.xlsx")

✅ All quote data has been written to ishowspeed_quotes.xlsx
